In [1]:
import torch
from tc_composer.func.affine_transform import AffineTransform
from tc_composer.func.activation import Activation
from tc_composer.func.merge import Sum

[INFO] tc_composer.settings - Setting default tensor type: torch.cuda.FloatTensor
[INFO] tc_composer.settings - Setting epsilon: 1e-16
[INFO] tc_composer.settings - Input tensor shape checking: False
[INFO] tc_composer.settings - Saving compiled options in: /home/ubuntu/tc_composer/options


In [2]:
batch_size = 2
in_n = 32
inp = torch.randn(batch_size, in_n)

out_n = 16

In [3]:
branch = AffineTransform(in_n, out_n) + AffineTransform(in_n, out_n)
merge = branch << Sum() << Activation('relu')

In [4]:
print(merge.tc_def(inp))

def Composition(
    float(2, 32) input,
    float(16, 32) weight,
    float(16) bias,
    float(16, 32) weight1,
    float(16) bias1
) -> (
    float(2, 16) output,
    float(2, 16) output1,
    float(2, 16) summed,
    float(2, 16) output2
)
{
    output(b, n) +=! input(b, i) * weight(n, i)
    output(b, n) = output(b, n) + bias(n)
    
    output1(b, n) +=! input(b, i) * weight1(n, i)
    output1(b, n) = output1(b, n) + bias1(n)
    
    summed(b, i) = output(b, i) + output1(b, i)
        where exists output(b, i), exists output1(b, i)
    
    output2(b, i) = fmax(summed(b, i), 0)
}


In [5]:
merge.recompile(inp)

[INFO] Composition - Compiling for input shape - [(2, 32)].
[WARNING] Composition - No option loaded from file for input shape - [(2, 32)].
[WARNING] Composition - Initializing naive options.


### Torch implementation

In [6]:
params = []
for t in merge.params:
    if t.dim() > 1:
        t = t.transpose(0, 1).contiguous()
    params.append(t)

In [8]:
from torch import nn, matmul, relu_ as relu_inplace

def mytorch(inp, params=params):
    weights = tuple(w for n,w in enumerate(params) if n % 2 == 0)
    biases = tuple(b for n,b in enumerate(params) if n % 2 == 1)

    out = matmul(inp, weights[0]).add_(biases[0]).add_(matmul(inp, weights[1])).add_(biases[1])

    return relu_inplace(out)

### Correctness

In [10]:
import numpy as np

np.testing.assert_allclose(merge(inp), mytorch(inp), rtol=1e-4)